In [1]:
%pwd

'c:\\Users\\ASUS\\Downloads\\MediGenie\\research'

In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\ASUS\\Downloads\\MediGenie'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader  #yha pr hmne pdf file use ki h , aur directory ke andr
from langchain.text_splitter import RecursiveCharacterTextSplitter #for chunking operation

In [5]:
# Extract Data From the PDF File

def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader ) #it will extend the info from pdf

    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf_file(data=r"C:\Users\ASUS\Downloads\MediGenie\Data")

In [7]:
# extracted_data                    # (we may see our data by running it)

In [8]:
# Now to perform the chunking operation, we use RecursiveCharacterTextSplitter
# Split the Data into Text Chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of text Chunks:", len(text_chunks))

Length of text Chunks: 5859


In [10]:
# text_chunks

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
# Download the embeddings from HuggingFace

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
from sentence_transformers import SentenceTransformer

c:\Users\ASUS\.conda\envs\medigenie-py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
embeddings= download_hugging_face_embeddings()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_19448\717155546.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [15]:
# now test the model whether it is able to convert my sentence into vector embedding and check ki
# ye hmara 384 vector dimension dera h ya ni (which we het from hugging face transfromer link?

query_result=embeddings.embed_query("Hello world")
print("Length",len(query_result))

Length 384


In [16]:
# query_result                 #to yha pr ye hme complete vector show krega

In [17]:
# now initialize the pinecon 

# on pinecone.ai hm apna cluster create kreinge to save all vectors and to create API Key 
# see it in .env file

In [18]:
#hm apna index manually bhi create kr skte h , but its time consuming so we load from data
# 
import os
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
# Ensure you have your Pinecone API key set as an environment variable
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [20]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define your index name
index_name = "medicalbot"

# Check if it already exists
existing_indexes = [index.name for index in pc.list_indexes()]
if index_name not in existing_indexes:
    # Only create if it doesn't exist
    pc.create_index(
        name=index_name,
        dimension=384,   #yha pr hme apne hugging model ka dimension size dena h
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists — skipping creation.")


Index 'medicalbot' created.


In [21]:
# now we may convert our chunks into vector embedding and store them in Pinecone

import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

In [22]:
import google.generativeai as genai
import os

# Set your Gemini API Key
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [23]:
from langchain_pinecone import PineconeVectorStore

# Assuming you have 'text_chunks', 'index_name', and 'embeddings' defined
# Embed each chunk and upsert the embeddings into your pinecone index to the pinecone vector database

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

print("Embeddings upserted into Pinecone index successfully.")

Embeddings upserted into Pinecone index successfully.


In [24]:
#yha hm notice kreinge ki hmaare starting me len(text_chunks)ka size tha pinecone vector db me store krne
# pr bhi same record count h

#and here it acts as our knowledge base

In [25]:
# Load Existing Index

from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your pinecone index

docsearch = PineconeVectorStore.from_existing_index(   #it will load all index in docsearch
    index_name=index_name,
    embedding=embeddings
)

print(f"Successfully loaded existing Pinecone index '{index_name}'.")

Successfully loaded existing Pinecone index 'medicalbot'.


In [26]:
docsearch

In [27]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})  #here it give 3 line responses

retrieved_docs = retriever.invoke("What is Acne?")

In [28]:
retrieved_docs  # yha pr hm chahte h ki hme aise ni proper definition ki form me result mile

[Document(id='9ac59ecc-be7b-4cef-a89d-3f753a1b8bf5', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\ASUS\\Downloads\\MediGenie\\Data\\Medical_book.pdf.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='4857d146-586d-4853-a5a3-ffb3df44a011', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\ASUS\\Downloads\\MediGenie\\Data\\Medical_book.pdf.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photogra

In [29]:
# But I want the proper definition instead of this type of output, so we insert the LLM Here

In [30]:
# from langchain_openai import OpenAI

import os
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY   # Replace with your actual API key

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)


In [31]:
# print(os.getenv("OPENAI_API_KEY")) 

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import os

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",   # or "gemini-pro"
    temperature=0,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)


In [33]:
system_prompt = (                                  #here i will give this prompt to my LLM
    "You are an assistant for question-answering tasks.\n"
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the " 
    "answer concise.\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(    # give complete output to the user input query
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [34]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


# model = genai.GenerativeModel('gemini-pro')
model = genai.GenerativeModel('models/gemini-1.5-flash')

response = model.generate_content("What is acne?")
# print(response.text)

# print(response['choices'][0]['message']['content'])

In [35]:
#now lets create our question answer chain
 
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain= create_retrieval_chain(retriever,question_answer_chain)

In [36]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

# response = rag_chain.invoke({"input": "What is Stats?"})        #it will generate I dont know bcoz
# print(response["answer"])                                     #it is outside to our knowledge base

Acne is a common skin disease characterized by pimples on the face, chest, and back.  It happens when skin pores become clogged with oil, dead skin cells, and bacteria.  Acne vulgaris is the medical term for common acne.


In [37]:
print(type(llm))

<class 'langchain_google_genai.chat_models.ChatGoogleGenerativeAI'>
